# 使用EFS作为SageMaker的训练数据输入

## 1 说明
本文为使用EFS作为SageMaker的训练数据输入。  
注意：该功能暂不能在中国区使用。

## 2 运行环境
Kernel 选择tensorflow2_p36或pytorch_p36均可。  
本文在boto3 1.17.99和sagemaker 2.45.0下测试通过。  

In [ ]:
import boto3,sagemaker
print(boto3.__version__)
print(sagemaker.__version__)

## 3 配置EFS
参考 https://docs.aws.amazon.com/zh_cn/efs/latest/ug/gs-step-two-create-efs-resources.html 配置EFS，注意配置合适的安全组。

## 4 获取/设置相关参数

In [ ]:
import boto3
import sagemaker
from sagemaker.image_uris import retrieve

sagemaker_session = sagemaker.Session()
iam = boto3.client('iam')
roles = iam.list_roles(PathPrefix='/service-role')
role=""
for current_role in roles["Roles"]:
    if current_role["RoleName"].startswith("AmazonSageMaker-ExecutionRole-"):
        role=current_role["Arn"]
        break
print(role)

注意事项：
- 1.SageMaker必须要有使用EFS的权限
- 2.确认EFS的安全组，允许SageMaker访问

In [ ]:
subnets = ["subnet-0eecdb20"]  # Should be same as Subnet used for FSx. Example: subnet-0f9XXXX
security_group_ids = ["sg-6478f13a"]  # Should be same as Security group used for FSx. sg-03ZZZZZZ
file_system_id = "fs-8eafd93a"  # FSx file system ID with your training dataset. Example: 'fs-0bYYYYYY'
efs_dir="/test" #EFS目录

In [ ]:
from sagemaker.inputs import FileSystemInput
file_system_directory_path = efs_dir
file_system_access_mode = "ro"#read only
file_system_type = "EFS"
train_fs = FileSystemInput(
    file_system_id=file_system_id,
    file_system_type=file_system_type,
    directory_path=file_system_directory_path,
    file_system_access_mode=file_system_access_mode,
)

## 5 训练

本文仅仅是列出了训练目录下的前100个文件，并没有实际训练，主要为演示获取EFS数据。

### 5.1 TensorFlow

In [ ]:
from sagemaker.tensorflow import TensorFlow

estimator = TensorFlow(
    base_job_name="tensorflow2-fsx-big",
    entry_point="ListFile.py",
    role=role,
    py_version="py37",
    framework_version="2.4.1",
    instance_count=1,
    instance_type="ml.m5.large",
    sagemaker_session=sagemaker_session,
    hyperparameters={"path":"/opt/ml/input/data/training"},
    subnets=subnets,
    security_group_ids=security_group_ids,
)
estimator.fit(train_fs)

### 5.2 PyTorch

In [ ]:
from sagemaker.pytorch import PyTorch

estimator = PyTorch(
    base_job_name="big-data-input",
    entry_point="ListFile.py",
    role=role,
    py_version="py36",
    framework_version="1.6.0",
    instance_count=1,
    instance_type="ml.m5.large",
    sagemaker_session=sagemaker_session,
    hyperparameters={"path":"/opt/ml/input/data/training"},
    subnets=subnets,
    security_group_ids=security_group_ids,
)
estimator.fit(train_fs)